In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import accuracy_score,roc_curve, auc, confusion_matrix, classification_report
from sklearn.metrics.pairwise import euclidean_distances, manhattan_distances, cosine_similarity

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

%matplotlib inline

### Get all pertenint data

In [7]:
nyoy = pd.read_csv("data/NyoyVolante_predicted_genres.csv")
nyoy['predicted_genre_name'].value_counts()

4    713
5    615
2    510
3    429
1     25
Name: predicted_genre_id, dtype: int64

In [ ]:
spotify = pd.read_csv('data/spotify_daily_charts_tracks_predicted_genres.csv')
spotify = spotify[spotify['predicted_genre_prob']>0.5]

In [ ]:
opm = pd.read_csv('data/OPM_playlist_tracks.csv')
spotifyopm = spotigy[spotify['']]

### Recommendation engine

In [ ]:
def recommend(seed_track):
    